In [1]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.append('../src')

In [ ]:
import yahoo_utils

game_id = 427
players = yahoo_utils.get_players(game_id)
current_schedule = yahoo_utils.get_games_by_week(game_id)
teams = yahoo_utils.get_teams(game_id)
all_matchups = yahoo_utils.get_all_matchups(game_id)

In [ ]:
import pandas as pd
preds = pd.read_csv('../data/2324_preds.csv').set_index(['playerId','gameId','playerTeam','opposingTeam','gameDate','week'])
y = pd.read_hdf('data/y.h5')
y_g = pd.read_hdf('data/y_g.h5')
y = pd.concat([y,y_g])


def weekify_stats(stats, schedule):
    full_daily_stats = []
    gdates = pd.to_datetime(stats.index.get_level_values('gameDate'), format='%Y%m%d')
    for week, games in schedule.items():
        games_df = pd.DataFrame(games)
        week_dates = pd.date_range(games_df.ts.min(), games_df.ts.max())
        for date in week_dates:
            daily_stats = stats[(gdates == date)].copy()
            daily_stats['date'] = date
            daily_stats['week'] = week
            full_daily_stats.append(daily_stats)
    return pd.concat(full_daily_stats)

daily_preds = weekify_stats(preds, current_schedule)
daily_preds = daily_preds.merge(players[['playerId','name', 'pos', 'player_key']], on='playerId', how='left')

y = y.merge(players[['playerId','name', 'pos', 'player_key']], on='playerId', how='left')

In [ ]:
from itertools import combinations, product, chain
from ast import literal_eval
import collections

def get_valid_lineups(day_teams):
    position_lookup = day_teams.set_index(['playerId'], drop=False).pos.apply(literal_eval).to_dict()
    n_combs = min(13, len(position_lookup))
    lineups = list(combinations(position_lookup.keys(), n_combs))

    def check_valid_lineup(lineup, position_lookup):
        positions_quota = {
            'D': 4,
            'C': 2,
            'RW': 2,
            'LW': 2,
            'G': 2,
            'WC': 1
        }
        positions_map = {k:[] for k in positions_quota}
        multi_positions = []
        for player in lineup:
            if len(position_lookup[player]) == 1:
                pos = position_lookup[player][0]
                if len(positions_map[pos]) == positions_quota[pos]:
                    if len(positions_map['WC']) == 0:
                        positions_map['WC'] = [player]
                    else:
                        return False
                else:
                    positions_map[pos] += [player]     
            else:
                multi_positions.append(player)

        positions_left = [p for p, v in positions_map.items() for i in range(positions_quota[p]-len(v))]
        multi_pos_positions = [position_lookup[p] for p in multi_positions]
        
        good_combinations = [c for c in list(product(*multi_pos_positions)) if collections.Counter(c) == collections.Counter(positions_left) or len(c) < 13]
        if len(good_combinations) == 0:
            return False
        return True
    
    # lineup_exp = []
    # for i, l in enumerate(lineups):
    #     if check_valid_lineup(l, position_lookup):
    #         lineup_df = day_teams.set_index('playerId').loc[list(l)].copy()
    #         lineup_df['lineup_id'] = i
    #         lineup_exp.append(lineup_df)
    return [l for l in lineups if check_valid_lineup(l, position_lookup)]

def get_expected_points(lineup, opp_lineup):
    from scipy import stats
    from process_data import PRED_COLS
    metrics = [m for metrics in PRED_COLS.values() for m in metrics]
    exp = lineup[metrics].sum()
    opp_exp = opp_lineup[metrics].sum()
    vals = {}
    for col in ['g','a','sog','fow','hit','block','pim','ppp', 'win', 'save', 'so']:
        vals[col] = stats.skellam.cdf(-1, opp_exp[col], exp[col]) + 0.5 * stats.skellam.pmf(0, opp_exp[col], exp[col])
    
    vals['plusmin'] = stats.skellam.cdf(-1, opp_exp['goalsfor']+exp['goalsaga'], opp_exp['goalsaga']+exp['goalsfor']) \
                    + 0.5 * stats.skellam.pmf(0, opp_exp['goalsfor']+exp['goalsaga'], opp_exp['goalsaga']+exp['goalsfor'])
    
    vals['gaa'] = stats.skellam.cdf(-1, exp['ga']/exp['icetime'], opp_exp['ga']/opp_exp['icetime'])\
                + 0.5 * stats.skellam.pmf(0, exp['ga']/exp['icetime'], opp_exp['ga']/opp_exp['icetime'])     
    return vals

In [ ]:
current_team_id = '427.l.21834.t.3'
team_matchups = [m for m in all_matchups if current_team_id in [t.team_key for t in m.teams]]
m = team_matchups[0]

print('WEEK', m.week)


opponent_id = [t.team_key for t in m.teams if t != current_team_id][0]
dates = pd.date_range(m.week_start, m.week_end)
daily_lineups = []
opp_daily_lineups = []

for date in dates:
    day_teams = teams.loc[pd.to_datetime(teams.index) == date]
    day_teams = teams.merge(daily_preds.reset_index(), on=['player_key', 'date'])
    daily_lineups.append(get_valid_lineups(day_teams[day_teams.team_id == current_team_id]))
    opp_daily_lineups.append(get_valid_lineups(day_teams[day_teams.team_id == opponent_id]))
    
all_lineups = product(product(*opp_daily_lineups), product(*opp_daily_lineups))
# for own_week_lineup, opp_week_lineup in all_lineups:
#     own_filter = None
#     opp_filter = None
#     for date, day_lineup in zip(dates, own_week_lineup):
#         day_filter = ((daily_preds.date == date)&(daily_preds.playerId.isin(day_lineup)))
#         if own_filter is None:
#             own_filter = day_filter
#         own_filter = own_filter | day_filter 
#     for date, day_lineup in zip(dates, opp_week_lineup):
#         day_filter = ((daily_preds.date == date)&(daily_preds.playerId.isin(day_lineup)))
#         if opp_filter is None:
#             opp_filter = day_filter
#         opp_filter = opp_filter | day_filter 
    
        
    # added_val = get_expected_points(lineup, opp_lineup)
# added_val


In [16]:
get_valid_lineups(day_teams.iloc[:12])

[(8478402,
  8479420,
  8475166,
  8478009,
  8474578,
  8476999,
  8477504,
  8481557,
  8475913,
  8477409,
  8480865,
  8477942)]

['C', 'RW', 'LW']

[('C', 'RW', 'LW'), ('LW', 'RW', 'C')]